<a href="https://colab.research.google.com/github/lucass373/Estudo-Machine-Learn/blob/main/predict_fraude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#Previsão de fraudes com machine learn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

In [10]:
#importar arquivo

df_credit = pd.read_csv('creditcard.csv')

In [11]:
df_credit.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [12]:
#verifica tipos
df_credit.dtypes

Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class       int64
dtype: object

In [13]:
df_credit.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [14]:
df_nao_fraude = df_credit.Amount[df_credit.Class == 0]
df_nao_fraude.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [15]:
#verifica os tipos de dados de fraude
df_fraude = df_credit.Amount[df_credit.Class == 1]
df_fraude.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [16]:
df_credit.Class.value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [17]:
#vrifica os tipos dos dados de fraude

df_fraude = df_credit[df_credit.Class == 1]

df_fraude

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
541,406.0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.00,1
623,472.0,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1
4920,4462.0,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,239.93,1
6108,6986.0,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,...,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,59.00,1
6329,7519.0,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,1.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


In [18]:
df_nao_fraude = df_credit[df_credit.Class == 0]

df_nao_fraude

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [19]:
#dataset com fraudes com a mesma quantidade de nao fraude

df_nao_fraudes = df_nao_fraude.sample(n=492)

df_nao_fraudes

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
232040,147018.0,1.553544,-2.193540,-0.582172,-0.457999,-1.545055,0.196104,-0.991765,0.095195,0.791631,...,0.112959,-0.221487,0.074597,0.569293,-0.493960,-0.419277,-0.029916,0.016375,315.00,0
261419,160008.0,1.949611,-1.357156,0.219427,-0.456660,-1.625814,0.267923,-1.601106,0.246553,0.304210,...,-0.096878,0.222435,0.224707,-0.398742,-0.680403,0.627034,0.015003,-0.034642,62.80,0
268721,163352.0,0.012222,0.033237,0.866112,0.240178,-0.787995,0.456246,0.294694,0.013549,-1.118998,...,-0.472650,-0.979973,0.322023,0.541148,-0.390687,-0.816467,-0.032702,-0.052076,144.00,0
227417,145067.0,-2.025219,2.140694,-0.456095,-1.036108,0.097165,-1.110647,1.276735,-0.593440,1.706571,...,-0.501134,-0.621224,0.134965,0.018451,-0.277384,0.022063,-0.393172,-0.247140,26.99,0
127769,78461.0,1.021277,-1.348771,1.519361,-0.246154,-1.987823,0.211811,-1.331368,0.220740,0.069733,...,0.323187,0.914532,-0.184495,0.623864,0.360933,-0.118037,0.054743,0.043550,132.41,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65761,51733.0,0.828945,-1.858542,1.254017,-0.170700,-1.988675,0.703872,-1.337365,0.324325,0.443218,...,0.306142,0.663056,-0.351559,0.083464,0.402608,-0.074524,0.035773,0.055133,243.50,0
219905,141930.0,2.014362,-0.117429,-1.010109,0.324591,-0.157585,-0.950014,0.061938,-0.176402,0.321513,...,-0.235475,-0.593831,0.343737,0.069106,-0.350871,0.173807,-0.076242,-0.069979,0.89,0
22588,32327.0,1.104669,0.125639,0.004413,0.396575,0.059126,-0.428185,0.341449,-0.079225,-0.535473,...,-0.269503,-0.898173,0.115258,0.034862,0.183504,0.143508,-0.057031,0.005483,54.90,0
196895,131745.0,1.819616,-1.536026,-3.112734,-1.873136,1.851454,3.044015,-0.488556,0.594176,-0.877366,...,0.173380,-0.136368,0.072246,0.711231,-0.104181,-0.315855,-0.067745,-0.032691,219.80,0


In [20]:
#concatenar os dois dataframes

df = pd.concat([df_nao_fraudes, df_fraude], axis = 0)
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
232040,147018.0,1.553544,-2.193540,-0.582172,-0.457999,-1.545055,0.196104,-0.991765,0.095195,0.791631,...,0.112959,-0.221487,0.074597,0.569293,-0.493960,-0.419277,-0.029916,0.016375,315.00,0
261419,160008.0,1.949611,-1.357156,0.219427,-0.456660,-1.625814,0.267923,-1.601106,0.246553,0.304210,...,-0.096878,0.222435,0.224707,-0.398742,-0.680403,0.627034,0.015003,-0.034642,62.80,0
268721,163352.0,0.012222,0.033237,0.866112,0.240178,-0.787995,0.456246,0.294694,0.013549,-1.118998,...,-0.472650,-0.979973,0.322023,0.541148,-0.390687,-0.816467,-0.032702,-0.052076,144.00,0
227417,145067.0,-2.025219,2.140694,-0.456095,-1.036108,0.097165,-1.110647,1.276735,-0.593440,1.706571,...,-0.501134,-0.621224,0.134965,0.018451,-0.277384,0.022063,-0.393172,-0.247140,26.99,0
127769,78461.0,1.021277,-1.348771,1.519361,-0.246154,-1.987823,0.211811,-1.331368,0.220740,0.069733,...,0.323187,0.914532,-0.184495,0.623864,0.360933,-0.118037,0.054743,0.043550,132.41,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


In [21]:
#ajustar index

df.reset_index(inplace = True)
df

,index,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,232040,147018.0,1.553544,-2.193540,-0.582172,-0.457999,-1.545055,0.196104,-0.991765,0.095195,...,0.112959,-0.221487,0.074597,0.569293,-0.493960,-0.419277,-0.029916,0.016375,315.00,0
1,261419,160008.0,1.949611,-1.357156,0.219427,-0.456660,-1.625814,0.267923,-1.601106,0.246553,...,-0.096878,0.222435,0.224707,-0.398742,-0.680403,0.627034,0.015003,-0.034642,62.80,0
2,268721,163352.0,0.012222,0.033237,0.866112,0.240178,-0.787995,0.456246,0.294694,0.013549,...,-0.472650,-0.979973,0.322023,0.541148,-0.390687,-0.816467,-0.032702,-0.052076,144.00,0
3,227417,145067.0,-2.025219,2.140694,-0.456095,-1.036108,0.097165,-1.110647,1.276735,-0.593440,...,-0.501134,-0.621224,0.134965,0.018451,-0.277384,0.022063,-0.393172,-0.247140,26.99,0
4,127769,78461.0,1.021277,-1.348771,1.519361,-0.246154,-1.987823,0.211811,-1.331368,0.220740,...,0.323187,0.914532,-0.184495,0.623864,0.360933,-0.118037,0.054743,0.043550,132.41,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
980,280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
981,280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
982,281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


In [22]:
#retirar dados para validação

df_val_nao_fraude = df.head(5)
df_val_nao_fraude


,index,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,232040,147018.0,1.553544,-2.193540,-0.582172,-0.457999,-1.545055,0.196104,-0.991765,0.095195,...,0.112959,-0.221487,0.074597,0.569293,-0.493960,-0.419277,-0.029916,0.016375,315.00,0
1,261419,160008.0,1.949611,-1.357156,0.219427,-0.456660,-1.625814,0.267923,-1.601106,0.246553,...,-0.096878,0.222435,0.224707,-0.398742,-0.680403,0.627034,0.015003,-0.034642,62.80,0
2,268721,163352.0,0.012222,0.033237,0.866112,0.240178,-0.787995,0.456246,0.294694,0.013549,...,-0.472650,-0.979973,0.322023,0.541148,-0.390687,-0.816467,-0.032702,-0.052076,144.00,0
3,227417,145067.0,-2.025219,2.140694,-0.456095,-1.036108,0.097165,-1.110647,1.276735,-0.593440,...,-0.501134,-0.621224,0.134965,0.018451,-0.277384,0.022063,-0.393172,-0.247140,26.99,0
4,127769,78461.0,1.021277,-1.348771,1.519361,-0.246154,-1.987823,0.211811,-1.331368,0.220740,...,0.323187,0.914532,-0.184495,0.623864,0.360933,-0.118037,0.054743,0.043550,132.41,0


In [23]:
df_val_fraude = df.tail(5)
df_val_fraude

,index,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
979,279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
980,280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
981,280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
982,281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1
983,281674,170348.0,1.991976,0.158476,-2.583441,0.408670,1.151147,-0.096695,0.223050,-0.068384,...,-0.164350,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,42.53,1


In [24]:
#retirar as 5 primeiras linhas
df = df.iloc[5:]

#retirar as ultimas linhas
df = df[:-5]

df

,index,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
5,186835,127274.0,2.028425,-0.028410,-1.845520,0.254048,0.495554,-0.411899,0.008912,-0.003682,...,-0.339138,-0.994056,0.294793,0.152672,-0.294906,0.175370,-0.077269,-0.044148,13.99,0
6,162398,115080.0,1.955162,-0.411396,-1.664663,0.186003,1.946557,3.969139,-1.128276,1.050656,...,0.219928,0.687659,0.181351,0.687131,-0.048044,-0.540485,0.077562,-0.037119,1.00,0
7,9550,14299.0,-1.960783,-1.021370,1.601870,-1.124639,1.917680,-1.689738,-0.680277,-0.039438,...,-0.334513,-0.850762,-0.202247,-0.482227,-0.510374,0.434790,0.010254,0.120473,11.99,0
8,82716,59519.0,-1.365043,-0.348198,1.748047,1.437430,-0.461558,-0.167825,-0.789295,0.800952,...,0.409590,0.684956,-0.003483,0.339530,-0.017965,-0.207099,-0.038387,-0.186686,74.80,0
9,26733,34225.0,-3.647636,0.423663,1.297429,-0.009270,0.756770,0.376760,-0.677921,-4.143221,...,3.101123,-1.487545,1.521918,-0.028958,0.500819,0.250130,0.725483,-0.480931,3.58,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,274382,165981.0,-5.766879,-8.402154,0.056543,6.950983,9.880564,-5.773192,-5.748879,0.721743,...,0.880395,-0.130436,2.241471,0.665346,-1.890041,-0.120803,0.073269,0.583799,0.00,1
975,274475,166028.0,-0.956390,2.361594,-3.171195,1.970759,0.474761,-1.902598,-0.055178,0.277831,...,0.473211,0.719400,0.122458,-0.255650,-0.619259,-0.484280,0.683535,0.443299,39.90,1
976,275992,166831.0,-2.027135,-1.131890,-1.135194,1.086963,-0.010547,0.423797,3.790880,-1.155595,...,-0.315105,0.575520,0.490842,0.756502,-0.142685,-0.602777,0.508712,-0.091646,634.30,1
977,276071,166883.0,2.091900,-0.757459,-1.192258,-0.755458,-0.620324,-0.322077,-1.082511,0.117200,...,0.288253,0.831939,0.142007,0.592615,-0.196143,-0.136676,0.020182,-0.015470,19.95,1


In [25]:
#concatenar o dataframe de validação

df_val_total = pd.concat([df_val_nao_fraude, df_val_fraude])

df_val_total.reset_index(inplace = True)

df_val_total_real = df_val_total.Class
df_val_total = df_val_total.drop(['level_0','index','Time','Class'], axis =1)

df_val_total


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,1.553544,-2.193540,-0.582172,-0.457999,-1.545055,0.196104,-0.991765,0.095195,0.791631,0.426019,...,0.539885,0.112959,-0.221487,0.074597,0.569293,-0.493960,-0.419277,-0.029916,0.016375,315.00
1,1.949611,-1.357156,0.219427,-0.456660,-1.625814,0.267923,-1.601106,0.246553,0.304210,0.928844,...,-0.447370,-0.096878,0.222435,0.224707,-0.398742,-0.680403,0.627034,0.015003,-0.034642,62.80
2,0.012222,0.033237,0.866112,0.240178,-0.787995,0.456246,0.294694,0.013549,-1.118998,0.657825,...,-0.431932,-0.472650,-0.979973,0.322023,0.541148,-0.390687,-0.816467,-0.032702,-0.052076,144.00
3,-2.025219,2.140694,-0.456095,-1.036108,0.097165,-1.110647,1.276735,-0.593440,1.706571,2.392023,...,0.664553,-0.501134,-0.621224,0.134965,0.018451,-0.277384,0.022063,-0.393172,-0.247140,26.99
4,1.021277,-1.348771,1.519361,-0.246154,-1.987823,0.211811,-1.331368,0.220740,0.069733,0.420243,...,0.356508,0.323187,0.914532,-0.184495,0.623864,0.360933,-0.118037,0.054743,0.043550,132.41
5,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,-5.587794,...,1.252967,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00
6,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,-3.232153,...,0.226138,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76
7,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,-3.463891,...,0.247968,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89
8,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,-5.245984,...,0.306271,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00
9,1.991976,0.158476,-2.583441,0.408670,1.151147,-0.096695,0.223050,-0.068384,0.577829,-0.888722,...,-0.017652,-0.164350,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,42.53


In [26]:
#verifica a distribuição dos dados do tipo fraude e nao fraude

df.Class.value_counts()

Class
0    487
1    487
Name: count, dtype: int64

In [27]:
#separar Labels e features

X = df.drop(['index', 'Time', 'Class'], axis = 1)
Y = df['Class']

In [28]:
print(X,Y)

           V1        V2        V3        V4        V5        V6        V7  \
5    2.028425 -0.028410 -1.845520  0.254048  0.495554 -0.411899  0.008912   
6    1.955162 -0.411396 -1.664663  0.186003  1.946557  3.969139 -1.128276   
7   -1.960783 -1.021370  1.601870 -1.124639  1.917680 -1.689738 -0.680277   
8   -1.365043 -0.348198  1.748047  1.437430 -0.461558 -0.167825 -0.789295   
9   -3.647636  0.423663  1.297429 -0.009270  0.756770  0.376760 -0.677921   
..        ...       ...       ...       ...       ...       ...       ...   
974 -5.766879 -8.402154  0.056543  6.950983  9.880564 -5.773192 -5.748879   
975 -0.956390  2.361594 -3.171195  1.970759  0.474761 -1.902598 -0.055178   
976 -2.027135 -1.131890 -1.135194  1.086963 -0.010547  0.423797  3.790880   
977  2.091900 -0.757459 -1.192258 -0.755458 -0.620324 -0.322077 -1.082511   
978 -1.374424  2.793185 -4.346572  2.400731 -1.688433  0.111136 -0.922038   

           V8        V9       V10  ...       V20       V21       V22  \
5  

In [29]:
#separar entre dados de treino e test

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42, stratify = Y)


In [30]:
#treinamento

lr = LogisticRegression(max_iter=1000)

lr.fit(X_train, Y_train)

pred = lr.predict(X_test)
acc = accuracy_score(Y_test, pred)

print(acc)

0.9282051282051282


In [31]:
#Validação
pred = lr.predict(df_val_total)

df = pd.DataFrame({'real': df_val_total_real, 'previsao': pred})

print(df)

   real  previsao
0     0         0
1     0         0
2     0         0
3     0         0
4     0         0
5     1         1
6     1         1
7     1         1
8     1         1
9     1         0
